### Морфологический анализатор pymorphy3

pymorphy3 (форк pymorphy2) написан на языке Python (работает под 2.7 и 3.5+). Он умеет:

* приводить слово к нормальной форме (например, “люди -> человек”, или “гулял -> гулять”).
* ставить слово в нужную форму. Например, ставить слово во множественное число, менять падеж слова и т.д.
* возвращать грамматическую информацию о слове (число, род, падеж, часть речи и т.д.)

При работе используется словарь OpenCorpora; для незнакомых слов строятся гипотезы. Библиотека достаточно быстрая: в настоящий момент скорость работы - от нескольких тыс слов/сек до > 100тыс слов/сек (в зависимости от выполняемой операции, интерпретатора и установленных пакетов); потребление памяти - 10…20Мб; полностью поддерживается буква ё.

(c) https://pymorphy2.readthedocs.io/en/stable/

Руководство пользователя расположено по ссылке: https://pymorphy2.readthedocs.io/en/stable/user/guide.html

In [ ]:
!pip install pymorphy3

In [1]:
import pymorphy3

Для морфологического анализа нужно инициализировать экземпляр класса `MorphAnalyzer` (лучше всего сделать это один раз в начале программы). У него есть метод `parse`, который возвращает список объектов типа `Parse`, каждый из которых представляет полный морфологический разбор словоформы.

Морфологический анализ &mdash; это определение характеристик слова на основе того, как это слово пишется. При морфологическом анализе не используется информация о соседних словах.

(c) https://pymorphy2.readthedocs.io/en/stable/user/guide.html

In [2]:
morph = pymorphy3.MorphAnalyzer()

# разбор слова
word = "стали"
parsed = morph.parse(word)
print(f"Разбор слова '{word}':")
for i, analysis in enumerate(parsed):
    print(f"{i+1}. {analysis}")

Разбор слова 'стали':
1. Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.975342, methods_stack=((DictionaryAnalyzer(), 'стали', 945, 4),))
2. Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.010958, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 1),))
3. Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.005479, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 6),))
4. Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 2),))
5. Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 5),))
6. Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), '

https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html

В pymorphy используются граммемы OpenCorpora с небольшими изменениями:

https://opencorpora.org/dict.php?act=gram

Структура разбора:

In [6]:
word = "бежал"
parsed_word = morph.parse(word)[0]  # Берем наиболее вероятный вариант

print(f"Слово: {parsed_word.word}")
print(f"Нормальная форма: {parsed_word.normal_form}")
print(f"Часть речи: {parsed_word.tag.POS}")
print(f"Падеж: {parsed_word.tag.case}")
print(f"Число: {parsed_word.tag.number}")
print(f"Время: {parsed_word.tag.tense}")
print(f"Полный тег: {parsed_word.tag}")
print(f"Склоняемость: {parsed_word.tag.animacy}")

Слово: бежал
Нормальная форма: бежать
Часть речи: VERB
Падеж: None
Число: sing
Время: past
Полный тег: VERB,perf,intr masc,sing,past,indc
Склоняемость: None


Работа с множественными разборами

In [ ]:
def analyze_word(word):
    analyses = morph.parse(word)
    print(f"Все варианты разбора слова '{word}':")
    for i, analysis in enumerate(analyses, 1):
        print(f"{i}. НФ: {analysis.normal_form:15} | "
              f"ЧР: {str(analysis.tag.POS):10} | "
              f"Вероятность: {analysis.score:.3f}")

# Тестируем на омонимах
analyze_word("ключ")
analyze_word("печь")
analyze_word("стекло")

Все варианты разбора слова 'ключ':
1. НФ: ключ            | ЧР: NOUN       | Вероятность: 0.769
2. НФ: ключ            | ЧР: NOUN       | Вероятность: 0.231
Все варианты разбора слова 'печь':
1. НФ: печь            | ЧР: NOUN       | Вероятность: 0.571
2. НФ: печь            | ЧР: INFN       | Вероятность: 0.286
3. НФ: печь            | ЧР: NOUN       | Вероятность: 0.143
Все варианты разбора слова 'стекло':
1. НФ: стекло          | ЧР: NOUN       | Вероятность: 0.690
2. НФ: стекло          | ЧР: NOUN       | Вероятность: 0.286
3. НФ: стечь           | ЧР: VERB       | Вероятность: 0.024


Вся грамматическая информация содержится в поле `tag`: там находится экземпляр класса `OpencorporaTag`, у которого названия грамматической категории будут тоже полями, а их значения &mdash; строками.

In [ ]:
def get_morph_features(word):
    parsed = morph.parse(word)[0]
    features = {
        'слово': parsed.word,
        'лемма': parsed.normal_form,
        'часть_речи': parsed.tag.POS,
        'падеж': parsed.tag.case,
        'число': parsed.tag.number,
        'род': parsed.tag.gender,
        'время': parsed.tag.tense,
        'залог': parsed.tag.voice,
        'наклонение': parsed.tag.mood
    }
    return {k: v for k, v in features.items() if v is not None}

# Примеры
words = ["столом", "писала", "красивая", "бежали"]
for word in words:
    features = get_morph_features(word)
    print(f"{word}: {features}")

столом: {'слово': 'столом', 'лемма': 'стол', 'часть_речи': 'NOUN', 'падеж': 'ablt', 'число': 'sing', 'род': 'masc'}
писала: {'слово': 'писала', 'лемма': 'писать', 'часть_речи': 'VERB', 'число': 'sing', 'род': 'femn', 'время': 'past', 'наклонение': 'indc'}
красивая: {'слово': 'красивая', 'лемма': 'красивый', 'часть_речи': 'ADJF', 'падеж': 'nomn', 'число': 'sing', 'род': 'femn'}
бежали: {'слово': 'бежали', 'лемма': 'бежать', 'часть_речи': 'VERB', 'число': 'plur', 'время': 'past', 'наклонение': 'indc'}


Базовая лемматизация

In [ ]:
def lemmatize_text(text):
    words = text.split()
    lemmas = []

    for word in words:
        # Убираем знаки препинания
        clean_word = ''.join(char for char in word if char.isalpha())
        if clean_word:
            parsed = morph.parse(clean_word)[0]
            lemmas.append(parsed.normal_form)

    return lemmas

text = "Машины ехали по дорогам, обгоняя друг друга"
lemmas = lemmatize_text(text)
print(f"Исходный текст: {text}")
print(f"Леммы: {lemmas}")

Исходный текст: Машины ехали по дорогам, обгоняя друг друга
Леммы: ['машина', 'ехать', 'по', 'дорога', 'обгонять', 'друг', 'друг']


Лемматизация с учетом признаков

In [ ]:
def smart_lemmatize(word, pos=None):
    analyses = morph.parse(word)

    if pos:
        # Фильтруем по части речи
        for analysis in analyses:
            if analysis.tag.POS == pos:
                return analysis.normal_form

    # Возвращаем наиболее вероятный вариант
    return analyses[0].normal_form

# Пример с омонимами
print(f"'стекло' как глагол: {smart_lemmatize('стекло', 'VERB')}")
print(f"'стекло' как существительное: {smart_lemmatize('стекло', 'NOUN')}")

'стекло' как глагол: стечь
'стекло' как существительное: стекло


Работа с грамматическими тегами в кириллице

In [ ]:
def detailed_analysis(word):
    parsed = morph.parse(word)[0]

    print(f"Детальный разбор слова '{word}':")
    print(f"Лемма: {parsed.normal_form}")
    print(f"Часть речи: {parsed.tag.POS}")
    print(f"Морфологические признаки:")

    # Все доступные признаки
    tag_dict = parsed.tag.cyr_repr  # Кириллическое представление
    print(tag_dict)

detailed_analysis("писавшему")

Детальный разбор слова 'писавшему':
Лемма: писать
Часть речи: PRTF
Морфологические признаки:
ПРИЧ,несов,неперех,прош,действ мр,ед,дт


Фильтрация по грамматическим признакам:

In [ ]:
def extract_by_pos(text, target_pos):
    words = text.split()
    result = []

    for word in words:
        clean_word = ''.join(char for char in word if char.isalpha())
        if clean_word:
            parsed = morph.parse(clean_word)[0]
            if parsed.tag.POS == target_pos:
                result.append({
                    'word': clean_word,
                    'lemma': parsed.normal_form,
                    'features': str(parsed.tag)
                })

    return result

text = "Рыжая кошка сидела на окне и смотрела на улицу"

print("Существительные:")
nouns = extract_by_pos(text, 'NOUN')
for noun in nouns:
    print(f"  {noun}")

print("\nГлаголы:")
verbs = extract_by_pos(text, 'VERB')
for verb in verbs:
    print(f"  {verb}")

Существительные:
  {'word': 'кошка', 'lemma': 'кошка', 'features': 'NOUN,anim,femn sing,nomn'}
  {'word': 'окне', 'lemma': 'окно', 'features': 'NOUN,inan,neut sing,loct'}
  {'word': 'улицу', 'lemma': 'улица', 'features': 'NOUN,inan,femn sing,accs'}

Глаголы:
  {'word': 'сидела', 'lemma': 'сидеть', 'features': 'VERB,impf,intr femn,sing,past,indc'}
  {'word': 'смотрела', 'lemma': 'смотреть', 'features': 'VERB,impf,tran femn,sing,past,indc'}


Поиск слов с определенными характеристиками (воспользуемся тем фактом, что наличие или отсутствие определённого признака у слова можно определить просто через оператор вхождения):

In [ ]:
parsed = morph.parse("кошки")[0]
print("NOUN" in parsed.tag)  # строка с одним признаком
print({"NOUN", "nomn"} in parsed.tag)  # множество из нескольких признаков

True
True


In [ ]:
def find_words_with_features(text, features):
    words = text.split()
    matches = []

    for word in words:
        clean_word = ''.join(char for char in word if char.isalpha())
        if clean_word:
            parsed = morph.parse(clean_word)[0]
            if features in parsed.tag:
                matches.append(parsed.word)

    return matches

text = "Лохматые собаки бежали по узкому тротуару"

# Поиск существительных в именительном падеже
features = {"NOUN", "nomn"} # множество
nominative_nouns = find_words_with_features(text, features)
print(f"Существительные в им.падеже: {nominative_nouns}")

Существительные в им.падеже: ['собаки']


Изменение падежей осуществляется с помощью метода `inflect()`, принимающего на вход множество граммем.

In [ ]:
def change_case(word, target_case):
    """Изменяет падеж слова"""
    parsed = morph.parse(word)[0]
    inflected = parsed.inflect({target_case})
    return inflected.word if inflected else word

words = ["кошка", "стол", "красивый", "быстро"]

for word in words:
    print(f"{word} ->:")
    for case in ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct']:
        new_form = change_case(word, case)
        print(f"  {case}: {new_form}")
    print()

кошка ->:
  nomn: кошка
  gent: кошки
  datv: кошке
  accs: кошку
  ablt: кошкой
  loct: кошке

стол ->:
  nomn: стол
  gent: стола
  datv: столу
  accs: стол
  ablt: столом
  loct: столе

красивый ->:
  nomn: красивый
  gent: красивого
  datv: красивому
  accs: красивого
  ablt: красивым
  loct: красивом

быстро ->:
  nomn: быстро
  gent: быстро
  datv: быстро
  accs: быстро
  ablt: быстро
  loct: быстро



Изменение числа

In [ ]:
def change_number(word, target_number):
    """Изменяет число слова"""
    parsed = morph.parse(word)[0]
    inflected = parsed.inflect({target_number})
    return inflected.word if inflected else word

words = ["кошка", "стол", "красивый", "бежал"]

for word in words:
    singular = change_number(word, 'sing')
    plural = change_number(word, 'plur')
    print(f"{word} -> ед.ч.: {singular}, мн.ч.: {plural}")

кошка -> ед.ч.: кошка, мн.ч.: кошки
стол -> ед.ч.: стол, мн.ч.: столы
красивый -> ед.ч.: красивый, мн.ч.: красивые
бежал -> ед.ч.: бежал, мн.ч.: бежали


Комбинированное изменение форм

In [11]:
def inflect_word(word, **grammemes):
    """Изменяет слово по заданным грамматическим признакам"""
    parsed = morph.parse(word)[0]
    
    inflected = parsed.inflect(set(grammemes.values()))
    return inflected.word if inflected else word

word = "интересная"
print(f"Исходное: {word}")

# Изменяем падеж и число
forms = [
    {'case': 'gent', 'number': 'sing'},  # родительный падеж, ед.ч.
    {'case': 'datv', 'number': 'plur'},  # дательный падеж, мн.ч.
    {'case': 'ablt', 'number': 'sing'},  # творительный падеж, ед.ч.
]

for gram_dict in forms:
    new_form = inflect_word(word, **gram_dict)
    print(f"{gram_dict} -> {new_form}")

verb = "делать"
print(f"Глагол: {verb}")

# Изменяем время, лицо, число
forms = [
    {'number': 'sing', 'person': '1per'},  # я делаю
    {'number': 'sing', 'person': '2per'},  # ты делаешь
    {'number': 'sing', 'person': '3per'},  # он делает
    {'number': 'plur', 'person': '1per'},  # мы делаем
    {'number': 'plur', 'person': '2per'},  # вы делаете
    {'number': 'plur', 'person': '3per'},  # они делают
]

for gram_dict in forms:
    new_form = inflect_word(verb, **gram_dict)
    print(f"{gram_dict} -> {new_form}")

Исходное: интересная
{'case': 'gent', 'number': 'sing'} -> интересной
{'case': 'datv', 'number': 'plur'} -> интересным
{'case': 'ablt', 'number': 'sing'} -> интересной
Глагол: делать
{'number': 'sing', 'person': '1per'} -> делаю
{'number': 'sing', 'person': '2per'} -> делаешь
{'number': 'sing', 'person': '3per'} -> делает
{'number': 'plur', 'person': '1per'} -> делаем
{'number': 'plur', 'person': '2per'} -> делаете
{'number': 'plur', 'person': '3per'} -> делают


Практический пример: склонение ФИО

In [14]:
def decline_name(full_name, case='gent'):
    """Склоняет ФИО в нужный падеж"""
    parts = full_name.split()
    declined_parts = []

    for part in parts:
        parsed = morph.parse(part)[0]
        declined = parsed.inflect({case})
        result = (declined.word if declined else part).capitalize()
        declined_parts.append(result)

    return ' '.join(declined_parts)

name = "Иванов Иван Иванович"
cases = ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct']
case_names = ['именительный', 'родительный', 'дательный', 'винительный', 'творительный', 'предложный']

for case, case_name in zip(cases, case_names):
    declined = decline_name(name, case)
    print(f"{case_name:15} -> {declined}")

именительный    -> Иванов Иван Иванович
родительный     -> Иванова Ивана Ивановича
дательный       -> Иванову Ивану Ивановичу
винительный     -> Иванова Ивана Ивановича
творительный    -> Ивановым Иваном Ивановичем
предложный      -> Иванове Иване Ивановиче


Получение возможных грамматических форм и проверка возможности форм


In [ ]:
def get_word_forms(word):
    """Получает все возможные формы слова"""
    parsed = morph.parse(word)[0]
    forms = set()

    for form in parsed.lexeme:
        forms.add(form.word)

    return sorted(forms)


def is_grammatical_form_possible(word, **grammemes):
    """Проверяет, возможна ли данная грамматическая форма"""
    parsed = morph.parse(word)[0]
    return parsed.inflect(set(grammemes.values())) is not None


word = "бежать"
forms = get_word_forms(word)
print(f"Все формы слова '{word}':")
for form in forms[:10]:  # Покажем первые 10 форм
    print(f"  {form}")

# Проверка возможности формы
test_cases = [
    {'number': 'plur', 'person': '1per'},
    {'gender': 'femn', 'tense': 'past'},
    {'case': 'gent', 'number': 'sing'}
]

for gram_dict in test_cases:
    possible = is_grammatical_form_possible(word, **gram_dict)
    print(f"Форма {gram_dict} возможна: {possible}")

Все формы слова 'бежать':
  беги
  бегите
  бегу
  бегут
  бегущая
  бегущего
  бегущее
  бегущей
  бегущем
  бегущему
Форма {'number': 'plur', 'person': '1per'} возможна: True
Форма {'gender': 'femn', 'tense': 'past'} возможна: True
Форма {'case': 'gent', 'number': 'sing'} возможна: True


Согласование слов с числительными

In [ ]:
butyavka = morph.parse('бутявка')[0]
nekuzyavaya = morph.parse('некузявая')[0]

print(f"1 {nekuzyavaya.make_agree_with_number(1).word} {butyavka.make_agree_with_number(1).word}")
print(f"2 {nekuzyavaya.make_agree_with_number(2).word} {butyavka.make_agree_with_number(2).word}")
print(f"6 {nekuzyavaya.make_agree_with_number(6).word} {butyavka.make_agree_with_number(6).word}")
print(f"121 {nekuzyavaya.make_agree_with_number(121).word} {butyavka.make_agree_with_number(121).word}")

print(f"12 {morph.parse('бятые')[0].make_agree_with_number(6).word} {morph.parse('пуськи')[0].make_agree_with_number(12).word}")


1 некузявая бутявка
2 некузявые бутявки
6 некузявых бутявок
121 некузявая бутявка
12 бятых пусек


### Задания для самостоятельного выполнения

#### Задание 1

Напишите функцию, которая принимает на вход прилагательное и существительное и определяет, согласованы ли они по роду, числу и падежу.

#### Задание 2

Напишите функцию, которая принимает на вход прилагательное и существительное и ставит прилагательное в форму, согласованную с существительным.

#### Задание 3

Напишите функцию, которая вычисляет &laquo;расстояние&raquo; между словами на основе их грамматических характеристик: количество отличающихся грамматических характеристик, деленное на общее число характеристик.

#### Задание 4

Напишите программу для вычисления лексического разнообразия текста. Оно вычисляется как отношение количества уникальных лексем в слове к количеству словоформ.

#### Задание 5

Напишите программу для вычисления по тексту:
* распределения частей речи
* распределения падежей существительных
* распределения времён глаголов
* любой другой интересующей вас информации.

Постройте соответствующие графики.

#### Задание 6

Напишите программу для поиска в тексте лексем, встретившихся по одному разу.

### Домашнее задание

1. Возьмите любой текст (например, новостной или скачать с wikisource или lib.ru)
2. Постройте словари слов: алфавитный список, частотный словарь, частотный словарь лексем.
3. Постройте алфавитный список всех глаголов. Каждый глагол поставьте в форму первого лица, единственного числа и настоящего времени, если глагол несовершенного вида, и будущего, если он совершенного вида.